
Connect to Google Drive for datasets (colab)

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Install dependencies (colab)

In [2]:
#!pip install simpleitk

Dependencies

In [3]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories <Br>
Comment out directory not in use


In [4]:
'''
#Toy Dataset Slices - Removed Empty Slices - Voxel Resampling [2, 0.6, 0.6] Bspline Interpolation 

train_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")
train_label_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_label_slices")

test_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_test_img_slices")
test_label_slice_dir= pathlib.Path(r"spider_toy_dset_slices/dummy_test_label_slices")
'''

#Full Dataset 
train_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/image_train")
train_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/label_train")

test_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/image_test")
test_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/label_test")

#Local Dataset Slices - Removed Slices
'''
train_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_slices")
train_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_slices")

test_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_slices")
test_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_slices")
'''
#Local Dataset Slices - Voxel Resampling [2, 0.6, 0.6] Bspline Interpolation 
'''
train_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_bspline_slices")
train_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_bspline_slices")

test_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_bspline_slices")
test_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_bspline_slices")
'''


'\ntrain_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_bspline_slices")\ntrain_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_bspline_slices")\n\ntest_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_bspline_slices")\ntest_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_bspline_slices")\n'

Image Slice Class

In [5]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri_Slice:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case

        #grabbing the image as well for downsampling testing
        #self.mri_mha = mri_mha
       
        #get 2d array from mri slice
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array
        
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Zero crop sanity check <br>
Testing Linspace and value mapping

In [6]:
''' #commenting out just have run all work for training
from transforms import array_transforms

test_image_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_img_slices/1_t1_30.mha")
test_label_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_label_slices/1_t1_30.mha")

#print(test_label_slice.mri_mha)

test_image_a = test_image_slice.hu_a
test_label_a = test_label_slice.hu_a

test_image_a, test_label_a = array_transforms.crop_zero(test_image_a, test_label_a)

test_image_crop_mha = sitk.GetImageFromArray(test_image_a)

test_label_tensor = torch.from_numpy(test_label_a)
        
test_label_tensor = test_label_tensor.to(torch.int)

val_range = np.arange(20)
print(val_range)
key_range = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 100, 201, 202, 203, 204, 205, 206, 207, 208, 209])

#pad to max resolution of slice in dset 
#label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max]) #torch.functional.pad takes care of cropping, no need to call array_transforms.crop_zero()

print(np.unique(test_label_a))

mapping_dict = dict(zip(key_range, val_range))

# Define a function to perform the mapping using the dictionary
def map_to_specified_set(value):
    return mapping_dict.get(value, np.nan)  # Return np.nan for values not found in the mapping_dict

# Create a vectorized version of the mapping function
value_map = np.vectorize(map_to_specified_set)

# Apply the mapping function to the array
mapped_array = value_map(test_label_a)

print("test arr after mapping", np.unique(mapped_array))
#sitk.WriteImage(test_image_crop_mha, "D:/sitk dump/1_t1_30_cropzero.mha")

#TODO continue here make images 1:1 scale and downsample to 64x64
#if(sitk_slice.GetSize()[0] != sitk_slice.GetSize()[1])
'''

' #commenting out just have run all work for training\nfrom transforms import array_transforms\n\ntest_image_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_img_slices/1_t1_30.mha")\ntest_label_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_label_slices/1_t1_30.mha")\n\n#print(test_label_slice.mri_mha)\n\ntest_image_a = test_image_slice.hu_a\ntest_label_a = test_label_slice.hu_a\n\ntest_image_a, test_label_a = array_transforms.crop_zero(test_image_a, test_label_a)\n\ntest_image_crop_mha = sitk.GetImageFromArray(test_image_a)\n\ntest_label_tensor = torch.from_numpy(test_label_a)\n        \ntest_label_tensor = test_label_tensor.to(torch.int)\n\nval_range = np.arange(20)\nprint(val_range)\nkey_range = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 100, 201, 202, 203, 204, 205, 206, 207, 208, 209])\n\n#pad to max resolution of slice in dset \n#label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max]) #torch.functional.pad takes care of cropping, n

Sort directories


In [7]:
#get lists from directories

#toy dset slices
image_path = train_img_slice_dir
label_path = train_label_slice_dir

image_dir_list = os.listdir(image_path)
label_dir_list = os.listdir(label_path)

print(image_path) 
print(label_path)

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(label_path))

print("train images count", dirlen)

#print(local_label_idr)


D:\Spider Slice Directories\image_train
D:\Spider Slice Directories\label_train
train images count 13275


Get max dimension of slice in dset for x y padding <br>
Images have slices with 0 label info removed and are cropped using zero crop <br>
Also get min max values of dset for tensor normalization <br>
<b> Results of runs in cell below this one <b> 

In [8]:
''' #commenting out just have run all work for training
from transforms import array_transforms

row_list = []
col_list = []

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = image_path.joinpath(image_dir_list[idx])
  lbl_path = label_path.joinpath(label_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  image = Mri_Slice(img_path)
  label = Mri_Slice(lbl_path)

  image_a = image.hu_a
  label_a = label.hu_a

  #crop zero
  image_a_cropzero, label_a_cropzero = array_transforms.crop_zero(image_a, label_a)

  #find array min max values for normalising in Dataset class
  if(idx ==0):
    
    image_tensor_min = np.min(image_a_cropzero)
    image_tensor_max = np.max(label_a_cropzero)
    label_tensor_min = np.min(label_a_cropzero)
    label_tensor_max = np.max(label_a_cropzero)
    
    unique_masks_a = np.unique(label_a)
  else:
    
    if(np.min(image_a_cropzero) < image_tensor_min):
      image_tensor_min = np.min(image_a_cropzero)
      image_tensor_min_dir = img_path
    if(np.min(label_a_cropzero) < label_tensor_min):
      label_tensor_min = np.min(label_a_cropzero)
    if(np.max(image_a_cropzero) > image_tensor_max):
      image_tensor_max = np.max(image_a_cropzero)
    if(np.max(label_a_cropzero) > label_tensor_max):
      label_tensor_max = np.max(label_a_cropzero)
    
  #find amount of unique masks for one-hot encoding 
    current_masks_a = np.unique(label_a)
    if(len(current_masks_a) > len(unique_masks_a)):
      unique_masks_a = current_masks_a
  #print("image res", image_a_cropzero.shape)
  
  
  row_list.append(image_a_cropzero.shape[0]) #add row value to list
  #print(image_a_cropzero.shape[0])
  col_list.append(image_a_cropzero.shape[1]) #add col value to list 
  
  

#calculate max 

row_dim_max = max(row_list)
col_dim_max = max(col_list)

row_dim_max = ((row_dim_max + 15) // 16) * 16 #nearest multiple of 16 
col_dim_max = ((col_dim_max + 15) // 16) * 16 #nearest multiple of 16

print("row max:", max(row_list))
print("col max:", max(col_list))

print("image tensor min", image_tensor_min)
print("image tensor max", image_tensor_max)
print("label tensor min", label_tensor_min)
print("label tensor max", label_tensor_max)

print("amount of masks", len(unique_masks_a))
print("masks array", unique_masks_a)



'''

' #commenting out just have run all work for training\nfrom transforms import array_transforms\n\nrow_list = []\ncol_list = []\n\nfor idx in range(0, dirlen):\n  #print("dirlen", dirlen)\n  \n #toy dset \n  \n  img_path = image_path.joinpath(image_dir_list[idx])\n  lbl_path = label_path.joinpath(label_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file \n\n  image = Mri_Slice(img_path)\n  label = Mri_Slice(lbl_path)\n\n  image_a = image.hu_a\n  label_a = label.hu_a\n\n  #crop zero\n  image_a_cropzero, label_a_cropzero = array_transforms.crop_zero(image_a, label_a)\n\n  #find array min max values for normalising in Dataset class\n  if(idx ==0):\n    \n    image_tensor_min = np.min(image_a_cropzero)\n    image_tensor_max = np.max(label_a_cropzero)\n    label_tensor_min = np.min(label_a_cropzero)\n    label_tensor_max = np.max(label_a_cropzero)\n    \n    unique_masks_a = np.unique(label_a)\n  else:\n    \n    if(np.min(image_a_cropzero) < i

Results from running above cell to save time

In [9]:
#Toy Dset Bspline Resample 
'''
row_dim_max = 448
col_dim_max = 224

image_tensor_min = -2057.0
image_tensor_max = 4292.0
label_tensor_min = 0.0
label_tensor_max = 208.0 #IVD 209 IS NOT EXISTENT NO CLUE WHY 
'''

'''
#Local Dset not resampled
row_dim_max = 928
col_dim_max = 400

image_tensor_min = -1000
image_tensor_max = 3096
label_tensor_min = 0.0
label_tensor_max = 209.0
'''

#Local dset bspline resample

row_dim_max = 448 #originally 464, less for memory constraints
col_dim_max = 224

image_tensor_min = -2230.0
image_tensor_max = 4385.0
label_tensor_min = 0.0
label_tensor_max = 209.0

unique_masks_a = np.array([0. ,   1. ,  2.   ,3.  , 4. ,  5. ,  6. ,  7. ,  8.,   9. ,100. ,201. ,202. ,203. ,204., 205. ,206. ,207. ,208. ,209.])


Value mapping for one-hot encoding label tensors (needed for BCE to work)

In [10]:
val_range = np.arange(20)
print(val_range)
key_range = unique_masks_a #no of unique values in labels, including BG 
print(key_range)

mapping_dict = dict(zip(key_range, val_range))

# Define a function to perform the mapping using the dictionary
def map_to_specified_set(value):
    return mapping_dict.get(value, np.nan)  # Return np.nan for values not found in the mapping_dict

# Create a vectorized version of the mapping function
value_map = np.vectorize(map_to_specified_set)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9. 100. 201. 202. 203.
 204. 205. 206. 207. 208. 209.]


Dataset Class

In [11]:
from transforms import tensor_transforms

class SpiderDataset(Dataset):
    def __init__(self, labels_dir, img_dir, transform=None, target_transform=None):
        self.labels_dir = labels_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.labels_dir))

    def __getitem__(self, idx):
        label_dir_list = os.listdir(self.labels_dir)
        image_dir_list = os.listdir(self.img_dir)

        image_dir_list = natsorted(image_dir_list)
        label_dir_list = natsorted(label_dir_list)

        img_path = self.img_dir.joinpath(image_dir_list[idx])
        label_path = self.labels_dir.joinpath(label_dir_list[idx])

        image = Mri_Slice(img_path)
        label = Mri_Slice(label_path)

        image_a = image.hu_a
        label_a = label.hu_a

        #value mapping label tensor 0-19
        label_a = value_map(label_a)

        image_tensor = torch.from_numpy(image_a)
        label_tensor = torch.from_numpy(label_a)
        
        image_tensor = image_tensor.to(torch.float32)
        label_tensor = label_tensor.to(torch.float32) #techincally these are ints not floats 

        #pad to max resolution of slice in dset 
        image_tensor = tensor_transforms.pad_to_resolution(image_tensor, [row_dim_max, col_dim_max])
        label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max]) #torch.functional.pad takes care of cropping, no need to call array_transforms.crop_zero()

        #normalise image tensor
        image_tensor = (image_tensor - image_tensor_min) / (image_tensor_max - image_tensor_min)
        #label_tensor = (label_tensor - label_tensor_min) / (label_tensor_max - label_tensor_min) #INTS HERE DO NOT NORMALISE 

        #one hot encoding label tensor BUG FIX
        #one hot label, this works only if the range on the label tensors is from 0 to 19 steps of 1 
        label_tensor = nn.functional.one_hot(label_tensor.long(), num_classes= 20) #resulting shape is torch.Size([8, 20, 448, 224]

        label_tensor = label_tensor.float()
        #print("tensor dims", image_tensor.shape)
        #print("label tensor min clamp",torch.min(label_tensor))
        #print("label tensor max clamp", torch.max(label_tensor))

        image_tensor = image_tensor.unsqueeze(0)
        #label_tensor = label_tensor.unsqueeze(0) 
        
        #print("label before permute shape", label_tensor.shape)
        #permute axes label tensor
        label_tensor = torch.permute(label_tensor, (2, 0, 1))

        #remove backround (0) channel in channel dim of label tensor 
        
        label_tensor = label_tensor[1:, :, :]  
        
        #print("tensor shape after removing first channel 2nd dim", label_tensor.shape)

        label_flattened = label_tensor.flatten()
        #print("label tensor unique values", torch.unique(label_flattened))

        image_tensor = image_tensor.to(device)
        label_tensor = label_tensor.to(device)

        #print(image_tensor.shape)
        return image_tensor, label_tensor




Dataset Class Instances


In [12]:
#toy train test dataset to test network running
#local_train_set = SpiderDataset(local_img_idr, local_label_idr)
dummy_train_set = SpiderDataset(train_label_slice_dir, train_img_slice_dir)

dummy_test_set = SpiderDataset(test_label_slice_dir, test_img_slice_dir)

print("train dataset len",dummy_train_set.__len__())
print("test dataset len",dummy_test_set.__len__())

train dataset len 13275
test dataset len 3187


Create Unet Instance







In [13]:
from models import unet 

input_channels = 1 #Hounsfield scale
output_channels = 19 #one for every class 0-9 vertebrae 10 spinal canal 11-19 ivd
start_filts = 32 #unet filters 
up_mode = 'upsample' #options are either 'upsample' for NN upsampling or 'transpose' for transpose conv

model = unet.UNet(in_channels= input_channels,num_classes=output_channels, start_filts=start_filts, up_mode=up_mode) #testing model hyperparams
model.to(device)
model.to(torch.float32)
#for param in model.parameters():
 #   print(param.device)

d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:214: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


UNet(
  (conv_final): Conv2d(32, 19, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): DownConv(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): DownConv(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3),

Load Model for resuming training

In [14]:

checkpoint= torch.load("D:/Spider Models Solvers Full Train/spider_model_20240512_133037_3")
model.load_state_dict(checkpoint['model_dict'])
model.to(device)
lr = 0.0001
optim = torch.optim.Adam(model.parameters(), lr=lr)
optim.load_state_dict(checkpoint['optimizer_dict'])


Hyperparameters


In [15]:
epochs = 4 #setting this to 3 epochs per training session takes about 6-8 hours
#$lr = 0.0001 #0.001 too large 
batchsize = 6 #max on local machine
loss_func = nn.BCEWithLogitsLoss() 
loss_func.to(device)
#optim = torch.optim.Adam(model.parameters(), lr=lr)

print(epochs)
print(lr)
print(batchsize)
print(loss_func)
print(start_filts)
print(up_mode)

print((loss_func))

4
0.0001
6
BCEWithLogitsLoss()
32
upsample
BCEWithLogitsLoss()


Dataloader

In [16]:
dummy_train_dataloader = DataLoader(dummy_train_set, batch_size = batchsize, shuffle=True)

dummy_test_dataloader = DataLoader(dummy_test_set, batch_size = batchsize, shuffle=True)


'''
for batch in dummy_train_dataloader:
    for tensor in batch:
        print("min", torch.min(tensor))
        print("max", torch.max(tensor))
'''

'\nfor batch in dummy_train_dataloader:\n    for tensor in batch:\n        print("min", torch.min(tensor))\n        print("max", torch.max(tensor))\n'

Tensor Dimensions Check

In [17]:
'''
for idx_slice in dummy_train_dataloader:
    for tensor in idx_slice:
        print(tensor.shape)
        break
'''

'\nfor idx_slice in dummy_train_dataloader:\n    for tensor in idx_slice:\n        print(tensor.shape)\n        break\n'

Accuracy Metrics

In [18]:
import metric

metric_calculator = metric.SegmentationMetrics(average=True, ignore_background=True,activation='sigmoid') 

metric_calculator_binary = metric.BinaryMetrics(activation='sigmoid') #for calculating spinal canal metrics since it's only 1 class


One Epoch <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [19]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    running_accu = 0.
    running_dice = 0.
    #running_precision = 0.
    #running_recall = 0.

    vert_running_accu = 0.
    vert_running_dice = 0.

    sc_running_accu = 0.
    sc_running_dice = 0.

    ivd_running_accu = 0.
    ivd_running_dice = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    #swap train dataloader for dset
    for i, data in enumerate(dummy_train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optim.zero_grad()

        
        # Make predictions for this batch
        outputs = model(inputs)

        #print("outputs shape", outputs.shape)
        
        #input output label shape [8 batchsize, 19 classes, row, col]


        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optim.step()

        #General metrics
        accu, dice, prec, recall = metric_calculator(labels, outputs)

        #Vertebrae Metrics (0-9)
        vert_labels = labels[:, :9, :, :]
        vert_outputs = outputs[:, :9, :, :]
        vert_accu, vert_dice, vert_prec, vert_recall = metric_calculator(vert_labels, vert_outputs)

        #Spinal Canal Metrics (10)
        sc_labels = labels[:, 10, :, :].unsqueeze(1)
        sc_outputs = outputs[:, 10, :, :].unsqueeze(1)
        sc_accu, sc_dice, sc_prec, sc_specificity, sc_recall = metric_calculator_binary(sc_labels, sc_outputs)

        #IVD Metrics (11-19)
        ivd_labels = labels[:, -9:, :, :]
        ivd_outputs = outputs[:, -9:, :, :]
        ivd_accu, ivd_dice, ivd_prec, ivd_recall = metric_calculator(ivd_labels, ivd_outputs)


        # Gather data and report
        #General
        running_loss += loss.item()
        running_accu += accu
        running_dice += dice
        #running_dice += dice
        #running_precision += prec
        #running_recall += recall

        #Vertebrae
        vert_running_accu += vert_accu
        vert_running_dice += vert_dice

        #Spinal Canal
        sc_running_accu += sc_accu
        sc_running_dice += sc_dice

        #IVD
        ivd_running_accu += ivd_accu
        ivd_running_dice += ivd_dice

        print("batch" ,i )
        #every 100 batches
        if i % 100 == 99: 
            last_loss = running_loss / 100 # loss per batch
            last_accu = running_accu / 100
            last_dice = running_dice / 100
            #last_precision = running_precision / 100
            #last_recall = running_recall / 100

            vert_last_accu = vert_running_accu / 100
            vert_last_dice = vert_running_dice /100

            sc_last_accu = sc_running_accu / 100
            sc_last_dice = sc_running_dice / 100

            ivd_last_accu = ivd_running_accu / 100
            ivd_last_dice = ivd_running_dice / 100

            #print('  batch {} loss: {}'.format(i + 1, last_loss))
            
            tb_x = epoch_index * len(dummy_train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            
            tb_writer.add_scalar('General/accuracy_train', last_accu, tb_x)
            tb_writer.add_scalar('General/dice_train', last_dice, tb_x)

            tb_writer.add_scalar('Vertebrae/accuracy_train', vert_last_accu, tb_x)
            tb_writer.add_scalar('Vertebrae/dice_train', vert_last_dice, tb_x)

            tb_writer.add_scalar('Spinal Canal/accuracy_train', sc_last_accu, tb_x)
            tb_writer.add_scalar('Spinal Canal/dice_train', sc_last_dice, tb_x)

            tb_writer.add_scalar('Intervertebral Discs/accuracy_train', ivd_last_accu, tb_x)
            tb_writer.add_scalar('Intervertebral Discs/dice_train', ivd_last_dice, tb_x)
            
            running_loss = 0.
            running_accu = 0.
            running_dice = 0.

            vert_running_accu = 0.
            vert_running_dice = 0.

            
            sc_running_accu = 0.
            sc_running_dice = 0.

            
            ivd_running_accu = 0.
            ivd_running_dice = 0.

    #print("loss", loss)
    return last_loss

Train Loop <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html


In [20]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#writer = SummaryWriter('runs/spider_seg_unet_epochs={}_lr={}_batchsize={}_loss=BCEWithLogits_startfilts={}_upmode={}'.format(epochs,lr, batchsize,start_filts,up_mode))
writer = SummaryWriter('runs/spider_full_batchsize_{}_lr_{}_trainses_2_{}'.format(batchsize, lr, timestamp))
epoch_number = 3 #updating every time for training resume 


best_vloss = 1_000_000.

for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    #print("avg loss in epoch", avg_loss)

    running_vloss = 0.0
    running_vaccu = 0.0
    running_vdice = 0.0

    vert_running_vaccu = 0.0
    vert_running_vdice = 0.0
    
    sc_running_vaccu = 0.0
    sc_running_vdice = 0.0

    ivd_running_vaccu = 0.0
    ivd_running_vdice = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(dummy_test_dataloader):
            vinputs, vlabels = vdata

            voutputs = model(vinputs)
            vloss = loss_func(voutputs, vlabels)
            
            vaccu, vdice, vprec, vrecall = metric_calculator(vlabels, voutputs)

            #Vertebrae Metrics (0-9)
            vert_vlabels = vlabels[:, :9, :, :]
            vert_voutputs = voutputs[:, :9, :, :]
            vert_vaccu, vert_vdice, vert_vprec, vert_vrecall = metric_calculator(vert_vlabels, vert_voutputs)

            #Spinal Canal Metrics (10)
            sc_vlabels = vlabels[:, 10, :, :].unsqueeze(1)
            sc_voutputs = voutputs[:, 10, :, :].unsqueeze(1)
            sc_vaccu, sc_vdice, sc_vprec, sv_vspecif, sc_vrecall = metric_calculator_binary(sc_vlabels, sc_voutputs)

            #IVD Metrics (11-19)
            ivd_vlabels = vlabels[:, -9:, :, :]
            ivd_voutputs = voutputs[:, -9:, :, :]
            ivd_vaccu, ivd_vdice, ivd_vprec, ivd_vrecall = metric_calculator(ivd_vlabels, ivd_voutputs)


            running_vloss += vloss
            running_vaccu += vaccu
            running_vdice += vdice

            vert_running_vaccu += vert_vaccu
            vert_running_vdice += vert_vdice

            sc_running_vaccu += sc_vaccu
            sc_running_vdice += vert_vdice

            ivd_running_vaccu += ivd_vaccu
            ivd_running_vdice += ivd_vdice



    avg_vloss = running_vloss / (i + 1)
    avg_vaccu = running_vaccu / (i + 1)
    avg_vdice = running_vdice / (i + 1)

    vert_avg_vaccu = vert_running_vaccu / (i + 1)
    vert_avg_vdice = vert_running_vdice / (i + 1)

    sc_avg_vaccu = sc_running_vaccu / (i + 1)
    sc_avg_vdice = sc_running_vdice / (i + 1)

    ivd_avg_vaccu = ivd_running_vaccu / (i + 1)
    ivd_avg_vdice = ivd_running_vdice / (i + 1)


    #print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    
    writer.add_scalars('Loss/train vs validation',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss},
                    epoch_number + 1)
    
    writer.add_scalars('General/accuracy_validation',
                    {'Accuracy': avg_vaccu},
                    epoch_number + 1)

    writer.add_scalars('General/dice_validation',
                    {'Dice': avg_vdice},
                    epoch_number + 1)
    #vert
    writer.add_scalars('Vertebrae/accuracy_validation',
                    {'Accuracy': vert_avg_vaccu},
                    epoch_number + 1)

    writer.add_scalars('Vertebrae/dice_validation',
                    {'Dice': vert_avg_vdice},
                    epoch_number + 1)
    #spinal canal
    writer.add_scalars('Spinal Canal/accuracy_validation',
                    {'Accuracy': sc_avg_vaccu},
                    epoch_number + 1)

    writer.add_scalars('Spinal Canal/dice_validation',
                    {'Dice': sc_avg_vdice},
                    epoch_number + 1)
    #ivd
    writer.add_scalars('Intervertebral Discs/accuracy_validation',
                    {'Accuracy': ivd_avg_vaccu},
                    epoch_number + 1)

    writer.add_scalars('Intervertebral Discs/dice_validation',
                    {'Dice': ivd_avg_vdice},
                    epoch_number + 1)
    


    writer.flush()
    
    
    # Track best performance, and save the model's state
   
    
        
    model_path = 'D:/Spider Models Solvers Full Train/spider_model_{}_{}'.format(timestamp, epoch_number)
    
    torch.save({'model_dict': model.state_dict(), 'optimizer_dict': optim.state_dict()}, model_path)
    
        
    epoch_number += 1

EPOCH 4:
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82
batch 83
batch 84
batch 85
batch 86
batch 87
batch 88
batch 89
batch 90
batch 91
batch 92
batch 93
batch 94
batch 95
batch 96
batch 97
batch 98
batch 99
batch 100
batch 101
batch 102
batch 103
batch 104
batch 105
batch 106
batch 107
batch 108
batch 109
b